Generates the following LUT:

LUT[N-bit, unsigned, fully integer binary number] = N-bit, unsigned, fully *fractional* binary number

In [1]:
import os
import itertools
import numpy as np
import copy
from tqdm import tqdm

# global variables
N_BITS = 8

MAX_VAL_OUT_BIN = "1"*N_BITS
MAX_VAL_OUT_FLOAT = 0
for i in range(N_BITS):
    MAX_VAL_OUT_FLOAT += 2**(-i-1)
    # print(f"MAX_VAL_OUT_FLOAT: {MAX_VAL_OUT_FLOAT}")

print(f"MAX_VAL_OUT_FLOAT: {MAX_VAL_OUT_FLOAT}")
np.random.seed(0)

MAX_VAL_OUT_FLOAT: 0.99609375


Helper functions

In [2]:
def generate_binary_numbers(n_bits):
    return np.array([''.join(bits) for bits in itertools.product('01', repeat=n_bits)])

def bin_to_int(bin_str):
    int_val = 0
    for i, bit in enumerate(bin_str[::-1]):
        int_val += int(bit)*(2**i)
    return int_val

def bin_to_frac(bin_str):
    frac_val = 0
    for i, bit in enumerate(bin_str):
        frac_val += int(bit)*(2**(-i-1))
    return frac_val

def frac_to_bin(frac):
    bin_str = ""
    for i in range(N_BITS):
        if frac >= 2**(-i-1):
            bin_str += "1"
            frac -= 2**(-i-1)
        else:
            bin_str += "0"
    return bin_str

## 1. Generate keys 

In [3]:
keys = generate_binary_numbers(N_BITS)

## 2. Generate values

In [4]:
reciprocal_LUT_dict = {}
for val_bin in keys:
    val_int = bin_to_int(val_bin)
    reciprocal_val_frac = 1/(val_int+1)
    reciprocal_val_bin = frac_to_bin(reciprocal_val_frac)
    reciprocal_LUT_dict[val_bin] = reciprocal_val_bin

## Write to verilog LUT module

In [5]:
rtl_dir = "04_ref_design"
template_fpath = os.path.join(rtl_dir, "division_LUT_template.sv")
with open(template_fpath, "r") as f:
    template = f.read()

In [6]:
print("00000000" in list(reciprocal_LUT_dict.values()))
print("00000001" in list(reciprocal_LUT_dict.values()))


False
True


In [7]:
module_rtl = template.replace("{N_BITS}", str(N_BITS))
module_rtl_top, module_rtl_bottom = module_rtl.split("// INSERT LUT HERE")
module_rtl_lines = [module_rtl_top]

for k,v in reciprocal_LUT_dict.items():
    line = f"        {N_BITS}'b{k} : reciprocal = {N_BITS}'b{v};\n"
    if v=="0"*(N_BITS-1)+"1":
        line = f"        default : reciprocal = {N_BITS}'b{v};\n"
        module_rtl_lines.append(line)
        break
        
    module_rtl_lines.append(line)

module_rtl_lines.append(module_rtl_bottom)

module_rtl = "".join(module_rtl_lines)

In [8]:
module_fpath = os.path.join(rtl_dir, f"udivision_LUT_{N_BITS}bit_int_to_{N_BITS}bit_frac.sv")
with open(module_fpath, "w") as f:
    f.write(module_rtl)